In [ ]:

import numpy as np
import pandas as pd
from pathlib import Path
import gc

# BOKEH
import bokeh.plotting as bk
import bokeh.models as bkmod
import bokeh.layouts as bklay
import bokeh.palettes as bkpalettes

bk.output_notebook()

# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp

# Custom imports
import bokeh_tools as bktools
import Presets as bkpresets

# BBStudies
import sys
sys.path.append('/Users/pbelanger/ABPLocal/BBStudies')
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as tune
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Base as phys
import BBStudies.Physics.Constants as cst



# Setting default values
#------------------------------------------------
# _default_fig_width  = 2000
_default_fig_width  = 1500
_default_fig_height = 400
_default_fig_pad    = 100



In [ ]:
collider = xt.Multiline.from_json('../001_configure_collider/zfruits/collider_001.json')
twiss = {}
twiss['lhcb1'] = collider['lhcb1'].twiss().to_pandas()
twiss['lhcb2'] = collider['lhcb2'].twiss().reverse().to_pandas()

light_twiss = {}
# Filtering twiss to get rid of slices
for sequence in ['lhcb1','lhcb2']:
    light_twiss[sequence] = xPlus.filter_twiss(twiss[sequence].set_index('name'),entries=['drift','..']).reset_index()

In [ ]:
sequence = 'lhcb1'
lattice_df = bktools.extract_lattice_info(collider[sequence],twiss[sequence])
bblr_info  = bktools.extract_bblr_info(collider[sequence],twiss[sequence])
collimator_info = bktools.extract_collimation_info(collider[sequence],twiss[sequence])

In [ ]:
sequence = 'lhcb1'
bblr_info  = bktools.extract_bblr_info(collider[sequence],twiss[sequence])
display(bblr_info[bblr_info.name.apply(lambda name: float(name.split('_')[-1])<3)])

sequence = 'lhcb2'
bblr_info  = bktools.extract_bblr_info(collider[sequence],twiss[sequence])
display(bblr_info[bblr_info.name.apply(lambda name: float(name.split('_')[-1])<3)])



considered_strength = []
for sequence in ['lhcb1','lhcb2']:
    bblr_info   = bktools.extract_bblr_info(collider[sequence],twiss[sequence])
    considered_strength.append(bblr_info.strength.abs().max())
ref_bblr = np.max(considered_strength)
ref_bblr

In [ ]:
bblr_info.groupby(bblr_info.name.apply(lambda name:name[7])).count()

In [ ]:
for IP,sub_bblr in bblr_info.groupby(bblr_info.name.apply(lambda name:name[7])):
    print(f"For IP{IP} :\n" + 40*'=')
    for _bblr in sub_bblr.name:
        print(collider[sequence].element_dict[_bblr].scale_strength)

In [ ]:
multipoles.groupby('type').get_group('dipole')

In [ ]:
[name for name in twiss['lhcb1'].name if '_den' in name]

In [ ]:
rich.inspect(collider['lhcb1'].element_dict['mbw.c6r3.b1_den'])

In [ ]:
light_twiss = {}
# Filtering twiss to get rid of slices
for sequence in ['lhcb1','lhcb2']:
    light_twiss[sequence] = xPlus.filter_twiss(twiss[sequence].set_index('name'),entries=['drift','..','_entry','_exit']).reset_index()

In [ ]:
import importlib
bkpresets = importlib.reload(bkpresets)
bktools = importlib.reload(bktools)

BOKEH_FIGS = {}

BOKEH_FIGS['twiss']   =  bkpresets.make_Twiss_Fig(collider,light_twiss,width=_default_fig_width,height=_default_fig_height)
BOKEH_FIGS['lattice'] =  bkpresets.make_LHC_Layout_Fig(collider,twiss,width=_default_fig_width,height=_default_fig_height)




In [ ]:
BOKEH_FIGS['lattice'].xaxis[1].visible = False
BOKEH_FIGS['twiss'].x_range = BOKEH_FIGS['lattice'].x_range

# BOKEH_FIGS['twiss'].frame_height = _default_fig_height
BOKEH_FIGS['twiss'].height = 3*_default_fig_height
BOKEH_FIGS['twiss'].min_border_bottom = 2*_default_fig_height
bk.show(bklay.column(BOKEH_FIGS['lattice'],BOKEH_FIGS['twiss']))

In [ ]:
sequence = 'lhcb1'
collimator_info = bktools.extract_collimation_info(collider[sequence],twiss[sequence])

In [ ]:
bbcw_group = collimator_info.groupby('type').get_group('BBCW')

ee_type = 'BBCW Down'
if ee_type == 'BBCW UP':
    group   = bbcw_group.loc[bbcw_group.name.apply(lambda name:name[4] in ['t','e'])]
else:
    group  = bbcw_group.loc[bbcw_group.name.apply(lambda name:name[4] in ['b','i'])]
group

In [ ]:
bbcw_group

In [ ]:
[name for name in twiss['lhcb1'].name if 'bbcw' in name]

In [ ]:
tw_data.groupby(tw_data.name.apply(lambda name:name.split('_')[0])).count()

In [ ]:
b_slots = np.ones
{  'ATLAS' : int(0/8 * (len(b_slots) * 2)),
'ALICE' : int(1/8 * (len(b_slots) * 2)),
'CMS'   : int(4/8 * (len(b_slots) * 2)),
'LHCB'  : int(7/8 * (len(b_slots) * 2)) - 3 }

In [ ]:
for (name,tw_group) in tw_data.groupby(tw_data.name.apply(lambda name:name.split('_')[0])):
    print(tw_group)

In [ ]:
tw_group.s.mean()

In [ ]:
tw_data.groupby(tw_data.name.apply(lambda name:name.split('_')[0])).s.max()

In [ ]:
test = 'tctpv.4l5.b1'

[_type_dict[key] for key in _type_dict.keys() if key in test][0]

In [ ]:
# def extract_lattice_info(line,twiss):

# Finding magnets by _order component of Mulitpole
_type_dict     = {'tcp.':'Primary','tcsg.':'Secondary','tcla.':'Absorber','tctp':'Tertiary','bbcw':'Wire'}
_length_dict   = {'Primary': 0.6, 'Secondary': 1.0, 'Absorber':1.0,'Tertiary': 1.0,'Wire':1.0} 
_strength_dict = {'Primary': 0.5, 'Secondary': 0.35, 'Absorber':0.2,'Tertiary': 0.2,'Wire':0.1}

tw = twiss['lhcb1']
# Creating lattice dictionary
lattice =  {}
lattice['name']    = []    
lattice['type']    = []
lattice['length']  = []
lattice['s']       = []
lattice['s_entry'] = []
lattice['s_exit']  = []
lattice['strength']= []

# Iterating through the elements
all_colls = []
for _type in _type_dict.keys():
    all_colls += [name for name in twiss['lhcb1'].name if _type in name]

# s_values       = pd.DataFrame({'s':line.get_s_elements(mode='upstream')},index=line.element_names).loc[all_multipoles[1],'s'].values
tw_data        = tw.set_index('name').loc[all_colls,['s']].reset_index()

# for ee,name,s in zip(*all_multipoles,s_values):
for (name,tw_group) in tw_data.groupby(tw_data.name.apply(lambda name:name.split('_')[0])):
    
    _type   = [_type_dict[key] for key in _type_dict.keys() if key in name][0]
    _length = _length_dict[_type]

    lattice['name'].append(name)
    lattice['type'].append(_type)
    lattice['length'].append(_length)
    lattice['s'].append(tw_group.s.min()+_length/2)
    lattice['s_entry'].append(tw_group.s.min())
    lattice['s_exit'].append(tw_group.s.min()+_length)
    lattice['strength'].append(_strength_dict[_type])

test = pd.DataFrame(lattice).sort_values(by='s').reset_index(drop=True)


In [ ]:
group = test.groupby('type').get_group('Wire')

In [ ]:
group

In [ ]:
group.groupby(group.name.apply(lambda name:'up' if name.split('bbcw')[1][0] in ['t','e'] else 'down')).count()

In [ ]:
[name for name in twiss['lhcb1'].name if 'bbcw' in name]

In [ ]:
[name for name in twiss['lhcb1'].name if 'tct' in name]

In [ ]:
collider['lhcb1'].element_dict['tctph.4l2.b1']

In [ ]:
bkpalettes.Category20c[20] + bkpalettes.Category20b[20]

In [ ]:
BOKEH_FIGS['twiss'].frame_height

# New axis function
#=====================================
def new_y_axis(fig,axis_name,side='none'):
    fig.extra_y_ranges[axis_name] = bkmod.Range1d(0,1)
    _ax = bkmod.LinearAxis(y_range_name=axis_name)
    if side == 'none':
        pass
    else:
        fig.add_layout(_ax,side)

    return _ax,axis_name

def new_x_axis(fig,axis_name,side='none'):
    fig.extra_x_ranges[axis_name] = bkmod.Range1d(0,1)
    _ax = bkmod.LinearAxis(x_range_name=axis_name)
    if side == 'none':
        pass
    else:
        fig.add_layout(_ax,side)

    return _ax,axis_name
#=====================================


# Creating Figure
#=====================================
fig = bk.figure(output_backend  = "webgl",
                height          = _default_fig_height, 
                width           = 2*_default_fig_width,
                title           = "Twiss parameters", 
                tools           = "box_zoom,pan,reset,save,hover,wheel_zoom",
                active_drag     = "box_zoom",
                active_scroll   = "wheel_zoom",
                toolbar_location= "right")

# No grid
# fig.grid.visible = False

# Saving tools to tags
fig.tags    = [ {str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':bkpalettes.Spectral11 + bkpalettes.PiYG11 + bkpalettes.RdPu9 + bkpalettes.RdGy11}]
# fig.tags[0]['BoxZoomTool'].update(dimensions = 'width')
fig.tags[0]['WheelZoomTool'].update(dimensions = 'height')
fig.tags[0]['HoverTool'].update(tooltips = [('Variable', '$name'),('s [m]','$x{0}'),(f'Value', '$y'),('Element','@name')])
#=====================================


# ii = 0
legends = {}
for beam,tw,ls,legend_colour in zip(['b1','b2'],[twiss['lhcb1'],twiss['lhcb2']],['solid',[3,1]],['blue','red']):
    source = bkmod.ColumnDataSource(tw.drop(columns=['W_matrix']))
    _line_list = []
    for key,color in zip(tw.columns.drop(['W_matrix','name','s']),fig.tags[1]['palette']):#tw.columns.drop(['W_matrix','name','s'])
        # fig.line(tw_b1['s'],tw_b1[key], line_width=2, color=color, alpha=0.8, legend_label=key)
        if (key in ['betx','bety'])&(beam=='b1'):
            _visible = True
        else:
            _visible = False
        _line = fig.line(x='s',y=key, line_width=2, color=color, alpha=0.8, line_dash=ls,name=key,legend_label=key,source=source,visible=_visible)
        _line_list.append((key,[_line]))

    # fig.legend.location = "top_left"
    legends[beam] = bkmod.Legend(items=_line_list,click_policy="hide",title=f'Beam {beam[-1]}')
    
    legends[beam].border_line_width = 2
    legends[beam].border_line_color = legend_colour
    legends[beam].border_line_alpha = 0.8

    # legends[beam].orientation = "horizontal"




fig.add_layout(legends['b1'], 'right')
fig.add_layout(legends['b2'], 'right')
# Removing original legend
fig.legend[-1].visible=False
# placing legend2 at the top

fig.xaxis.axis_label = "Position, s [m]"
fig.yaxis.axis_label = "Value"
fig.y_range.only_visible = True
bk.show(fig)

In [ ]:
fig.y_range.only_visible

In [ ]:
tw.drop(columns=['W_matrix'])

In [ ]:
tw

In [ ]:





# Creating Figure
#=====================================
fig = bk.figure(output_backend  = "webgl",
                height          = _default_fig_height, 
                width           = _default_fig_width,
                title           = "Lattice", 
                tools           = "box_zoom,pan,reset,save,hover,wheel_zoom",
                active_drag     = "box_zoom",
                active_scroll   = "wheel_zoom",
                toolbar_location= "right")

# No grid and y axis but adding 0 line
fig.grid.visible = False



# Saving tools to tags
fig.tags    = [ {str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':bkpalettes.Spectral11 + bkpalettes.PiYG11 + bkpalettes.RdPu9 + bkpalettes.RdGy11}]
fig.tags[0]['BoxZoomTool'].update(dimensions = 'width')
fig.tags[0]['WheelZoomTool'].update(dimensions = 'height')
fig.tags[0]['HoverTool'].update(tooltips = [('Element','@name'),('s [m]','$x{0}'),('Magnet','@type')])
#=====================================


for sequence,baseline,beam_color in zip(['lhcb1','lhcb2'],[1.5,-1.5],['blue','red']):
    beam       = sequence[-2:]
    lattice_df = bktools.extract_lattice_info(collider[sequence],twiss[sequence])
    bblr_info  = bktools.extract_bblr_info(collider[sequence],twiss[sequence])
    tw         = twiss[sequence]

    # Adding 0 line
    fig.hspan(y=[baseline],line_width=[1], line_color="black")
    fig.hspan(y=[-1.05+baseline,1.05+baseline],line_width=[2,2], line_color=beam_color)
    fig.varea(x=[tw.s.min(),tw.s.max()],y1=[-1.05+baseline,-1.05+baseline], y2=[1.05+baseline,1.05+baseline], alpha=0.05,fill_color=beam_color)   

    # Making step function for each element
    for ee_type,color in zip(['dipole','quadrupole','sextupole','octupole','bblr'],['royalblue','firebrick','forestgreen','darkmagenta','black']):
        
        if ee_type != 'bblr':
            group   = lattice_df.groupby('type').get_group(ee_type)
            legend_label = ee_type.capitalize()
        else:
            # Adding BBLR contribution
            group = bblr_info.rename(columns={'strength':'knl'})
            legend_label = ee_type.upper()

        element_line_x = [[_entry,_entry,_exit,_exit] for _entry,_exit in zip(group.s_entry,group.s_exit)] 
        element_line_y = [[0,_knl,_knl,0] for _knl in group.knl] 

        element_df = pd.DataFrame({'x':np.array(element_line_x).flatten(),'y':np.array(element_line_y).flatten()})    
        element_line_x = np.array(element_line_x).flatten()
        element_line_y = np.array(element_line_y).flatten()
        element_baseline = np.zeros(len(element_line_x))

        if ee_type == 'dipole':
            element_line_y   = 0.2*np.divide(element_line_y,element_line_y,out=np.zeros(len(element_line_y)),where=element_line_y!=0)
            element_baseline = -element_line_y 
        else:
        # if ee_type == 'quadrupole':
            element_line_y   = element_line_y/np.max(np.abs(element_line_y))

        source = bkmod.ColumnDataSource(pd.DataFrame({'name':np.repeat(group.name,4),'type':np.repeat(group.type,4),'x':element_line_x,'y':element_line_y+baseline,'baseline':element_baseline+baseline}))
        _varea = fig.varea(x='x',y1='baseline', y2='y', alpha=0.6,fill_color=color,source=source,legend_label=legend_label)
        
        if ee_type in ['sextupole','octupole','bblr']:
            fig.circle(group.s,group.knl/np.max(np.abs(group.knl)) + baseline, size=3, color=color, alpha=0.5,legend_label=legend_label)




    if sequence == 'lhcb1':

        fig.legend.location    = "top_left"
        fig.legend.click_policy= "hide"


        # Adding  IP locations


        sequence = 'lhcb1'
        IPs    = tw.loc[tw.name.isin([f'ip{i}' for i in range(1,8+1)]),['name','s']]
        arcs_s = tw.loc[tw.name.isin([f's.arc.{arc}.{beam}' for arc in [f'{np.roll(range(1,9),-i)[0]}{np.roll(range(1,9),-i)[1]}' for i in range(0,8)]]),['name','s']]
        arcs_e = tw.loc[tw.name.isin([f'e.arc.{arc}.{beam}' for arc in [f'{np.roll(range(1,9),-i)[0]}{np.roll(range(1,9),-i)[1]}' for i in range(0,8)]]),['name','s']]
        arc_mids = pd.DataFrame({'name':[f'ARC {arc}' for arc in arcs_s.sort_values('name').name.apply(lambda arc: arc.split('.')[2])],
                                's':(arcs_s.sort_values('name').s.values + arcs_e.sort_values('name').s.values)/2})

        IPs.loc[:,'name'] = IPs.name.str.upper()
        _label_overrides = IPs.set_index('s')['name'].to_dict()
        _label_overrides.update(arc_mids.set_index('s')['name'].to_dict())


ax,axis_name = bktools.new_x_axis(fig,axis_name='IPs',side='above')
fig.extra_x_ranges[axis_name] = fig.x_range
fig.xaxis[0].ticker = sorted(list(_label_overrides.keys()))
fig.xaxis[0].major_label_overrides = _label_overrides
fig.xaxis[0].major_tick_line_width = 3
fig.xaxis[0].major_tick_in = 5

# fig.yaxis.visible = False
fig.yaxis[0].ticker = [1.5,-1.5]
fig.yaxis[0].major_label_overrides = {1.5:'Beam 1',-1.5:'Beam 2'}



# all_mb = [s for s,name in zip(lattice_df.s,lattice_df.name) if 'mb' in name]

# fig.line(element_df.x,element_df.y, alpha=0.6)
# fig.line(element_line_x,element_line_y, alpha=0.6)

bk.show(fig)

In [ ]:
_label_overrides = IPs.set_index('s')['name'].to_dict()
_label_overrides.update(arcs_s.set_index('s')['name'].to_dict())
_label_overrides.update(arcs_e.set_index('s')['name'].to_dict())

In [ ]:
IPs.name.str.upper()

In [ ]:
arc_mids

In [ ]:
arcs_s

In [ ]:
arcs_s

In [ ]:
_label_overrides

In [ ]:
beam='b1'


In [ ]:
for i in range(0,8):
    arc = f'{np.roll(range(1,9),-i)[0]}{np.roll(range(1,9),-i)[1]}'
    
    thisArc = twiss.set_index('name').loc[f's.arc.{arc}.b1':f'e.arc.{arc}.b1']

In [ ]:
thisArc

In [ ]:
rich.inspect(ee)

In [ ]:
line.particle_ref.p0c

In [ ]:
math.factorial(np.arange(12+1))

In [ ]:
gamma0 = 1/np.sqrt(1-ee.beta0**2)
gamma0

In [ ]:
6.8e12/line.particle_ref.mass0

In [ ]:
gamma0 = 1/np.sqrt(1-ee.beta0**2) 
E      = gamma0*xp.PROTON_MASS_EV
p0c    = ee.beta0*E


In [ ]:
gamma0 = 1/np.sqrt(1-ee.beta0**2) 
E      = gamma0*xp.PROTON_MASS_EV
p0c    = ee.beta0*E
p0c

In [ ]:
line.particle_ref.p0c[0]

In [ ]:
float(line.particle_ref.p0c)

In [ ]:
xp.PROTON_MASS_EV

In [ ]:
line.particle_ref.mass0

In [ ]:
np.sqrt((6.8e12)**2-line.particle_ref.mass0**2)

In [ ]:
np.sqrt((6.8e12)**2-line.particle_ref.mass0**2)

In [ ]:
np.sqrt(1/(1-ee.beta0**2) - 1)*line.particle_ref.mass0

In [ ]:
ee.ref_shift_x - ee.other_beam_shift_x

In [ ]:

import scipy.special as sciSpec

def bblr_knl(ee_bb,dx,dy):
    Nb    = ee_bb.n_particles
    IL_eq = Nb*cst.elec*cst.c

    gamma0 = 1/np.sqrt(1-ee_bb.beta0**2) 
    E      = gamma0*xp.PROTON_MASS_EV
    p0     = ee.beta0*E/cst.c

    
    n = np.arange(12+1)
    integratedComp = -cst.mu0*(IL_eq)*sciSpec.factorial(n)/(2*np.pi)/(dx+1j*dy)**(n+1)
    _kn,_sn = np.real(integratedComp),np.imag(integratedComp)
    
    knl,snl = _kn/p0,_sn/p0
    return  knl,snl


def compute_bblr_strength(ee_bb,x,y,betx,bety):
    # Fixing exmittance since strength should be normalized at the end
    emittxy = [1,1]

    # Computing beam separation
    dx = x - ee_bb.ref_shift_x - ee_bb.other_beam_shift_x
    dy = y - ee_bb.ref_shift_y - ee_bb.other_beam_shift_y

    # Computing knl of interaction
    knl,snl = bblr_knl(ee_bb,dx,dy)
    
    # Computing tune shift
    vec_J = {}
    for plane,amplitudes in zip(['x','y'],[[1,0],[0,1]]):
        ax,ay = amplitudes
        DQx,DQy = tune.DQx_DQy_octupole(ax,ay,  betxy   = [betx,bety],
                                                emittxy = emittxy,
                                                k1l     = 0,
                                                k3l     = knl[3])
    
        vec_J[plane] = np.array([DQx,DQy])


    # Strength
    #---------------------------------------
    area   = np.abs(np.cross(list(vec_J['x']), list(vec_J['y']))/2)
    strength = area
    #---------------------------------------

    return strength



# def extract_bblr_info(line):


sequence   = 'lhcb2'
beam       = sequence[-2:]
# lattice_df = bktools.extract_lattice_info(collider[sequence])
# bblr_info  = bktools.extract_bblr_info(collider[sequence],twiss[sequence])
line = collider[sequence]
tw   = twiss[sequence]

# Creating lattice dictionary
lattice =  {}
lattice['name']    = []    
lattice['type']    = []
lattice['length']  = []
lattice['s']       = []
lattice['s_entry'] = []
lattice['s_exit']  = []
lattice['strength']= []

# Iterating through the elements
all_bblr = line.get_elements_of_type(xf.beam_elements.beambeam2d.BeamBeamBiGaussian2D)
tw_data  = tw.set_index('name').loc[all_bblr[1],['s','x','y','betx','bety']]
beam     = all_bblr[1][0].split('_')[1][-2:]
if beam.lower() == 'b2':
    tw_data.loc[:,'x'] = -tw_data['x']

for ee,(name,tw_row) in zip(all_bblr[0],tw_data.iterrows()):
    # print(name,ee.scale_strength,tw_row.s)

    strength = compute_bblr_strength(ee,tw_row.x,tw_row.y,tw_row.betx,tw_row.bety)
    length   = 7.5/10
    lattice['name'].append(name)
    lattice['type'].append('bblr')
    lattice['length'].append(length)
    lattice['s'].append(tw_row.s + length/2)
    lattice['s_entry'].append(tw_row.s)
    lattice['s_exit'].append(tw_row.s + length)
    lattice['strength'].append(strength)


bblr_info =  pd.DataFrame(lattice).sort_values(by='s').reset_index(drop=True)




In [ ]:
all_bblr[1][0].split('_')[1][-2:]

In [ ]:
bblr_info.groupby(bblr_info.name.apply(lambda name: name.split('.')[1][:2]))['strength'].mean()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(bblr_info.s,bblr_info.strength,'-o')
plt.xlim([6500,6800])
plt.axhline(0)

In [ ]:
bblr_info

In [ ]:
collider['lhcb1'].element_dict['bb_lr.l5b1_25']

In [ ]:
type(collider['lhcb1'].element_dict['bb_lr.l5b1_01']) == 

In [ ]:
xf.beam_elements.beambeam2d.BeamBeamBiGaussian2D

In [ ]:
xt.beam_elements.beambeam2d

In [ ]:
[name for name in tw_b1.name if 'bb_lr' in name]

In [ ]:
group.set_index('s').loc[11749:11755]

In [ ]:
group.knl

In [ ]:
pd.DataFrame({'name':np.repeat(group.name,4),'type':np.repeat(group.type,4),'x':element_line_x,'y':element_line_y,'baseline':element_baseline})

In [ ]:
group  = lattice_df.groupby('type').get_group('dipole')
group.groupby(group.name.apply(lambda name: name.split('.')[0]))['length'].mean()

In [ ]:
group.set_index('s').loc[260:285]

In [ ]:
element_df

In [ ]:
list(element_line_y)

In [ ]:
list(element_line_x)

In [ ]:
group.loc[group.name.str.contains('mb.')]

In [ ]:
# Making step function for each element
group  = lattice_df.groupby('type').get_group('dipole')
element_line_x = [[_entry,_entry,_exit,_exit] for _entry,_exit in zip(group.s_entry,group.s_exit)] 
element_line_y = [[0,_knl,_knl,0] for _knl in group.knl] 

element_line_x = np.array(element_line_x).flatten()
element_line_y = np.array(element_line_y).flatten()

In [ ]:
np.divide(element_line_y,element_line_y,out=np.zeros(len(element_line_y)),where=element_line_y!=0)

In [ ]:

dipole_line_x = [[_entry,_entry,_exit,_exit] for _entry,_exit in zip(lattice_df.s_entry,lattice_df.s_exit)] 
dipole_line_x = [x for row in dipole_line_x for x in row]
dipole_line_x

In [ ]:
group  = lattice_df.groupby('type').get_group('dipole')
element_line_x = [[_entry,_entry,_exit,_exit] for _entry,_exit in zip(group.s_entry,group.s_exit)] 
element_line_y = [[0,_knl,_knl,0] for _knl in group.knl] 

In [ ]:
np.array(element_line_x).flatten()

In [ ]:
np.array([x for row in element_line_x for x in row]) == 

In [ ]:
[**dipole_line_x for ee in dipole_line_x]

In [ ]:
np.repeat(lattice_df.s_entry,2)

In [ ]:
np.repeat(lattice_df.s_entry,2)
np.tile([0,1,1,0],len(lattice_df))

In [ ]:

dipole_line_x = [[_entry,_entry,_exit,_exit] for _entry,_exit in zip(lattice_df.s_entry,lattice_df.s_exit)] 
dipole_line_x = [x for row in dipole_line_x for x in row]
dipole_line_x

In [ ]:
len(dipole_line_y)

In [ ]:
_line.element_dict['mqwa.a4r3.b1..4']

In [ ]:
tw.groupby(tw.name.apply(lambda ee: ee.split('..')[0])).get_group('mqwa.a4r3.b1')

In [ ]:



beam = 'b1'
_line   = collider[f'lhc{beam}']
lattice_df= extract_lattice_info(_line)
   

In [ ]:
full_lattice.groupby(full_lattice.name.apply(lambda name: name.split('..')[0])).get_group('mqwa.a4r3.b1')

In [ ]:
full_lattice

In [ ]:
lattice_df

In [ ]:
group = lattice_df.groupby(lattice_df.name.apply(lambda name: name.split('..')[0])).get_group('mqwa.a4r3.b1')

In [ ]:
pd.DataFrame({'s':_line.get_s_elements()},index=_line.element_names).loc[all_multipoles[1],'s'].values

In [ ]:
tw.set_index('name').loc[all_multipoles,'s']

In [ ]:
all_multipoles = _line.get_elements_of_type(xt.elements.Multipole)[1]

In [ ]:
[ name for s,name in zip(_line.get_s_elements(),_line.element_names) if name in all_multipoles]

In [ ]:
_line.element_names

In [ ]:

lattice_df

In [ ]:
_line.get_s_elements()

In [ ]:
_line.get_elements_of_type(xt.elements.Multipole)[1]

In [ ]:
_line.get_s_elements()

In [ ]:
collider['lhcb1'].element_dict['mb.b8r3.b1..1_entry']

In [ ]:
beam = 'b1'
tw   = tw_b1
# Extracting magnet strengths:
_line   = collider[f'lhc{beam}']
lattice =  {}
lattice['s']   = []
lattice['name']   = []
lattice['dipole']     = []
lattice['quadrupole'] = []
lattice['sextupole']  = []
lattice['octupole']   = []
for name,s in zip(tw.name,tw.s):
    if name in _line.element_dict.keys():
        ee = _line.element_dict[name]

    if type(ee) is xt.elements.Multipole:
        _knl = ee.knl[:4]
        _knl = np.pad(_knl, (0, 4-len(_knl)), 'constant', constant_values=np.nan)
        lattice['s']      +=  [s]
        lattice['name']   +=  [name] 
        # lattice['knl'] = _knl
        lattice['dipole']     += [_knl[0]]
        lattice['quadrupole'] += [_knl[1]]
        lattice['sextupole']  += [_knl[2]]
        lattice['octupole']   += [_knl[3]]

lattice_df = pd.DataFrame(lattice)
lattice_df.sort_values(by='s',inplace=True)
lattice_df.loc[lattice_df['dipole']==0,'dipole'] = np.nan

In [ ]:
collider[f'lhc{beam}']

In [ ]:
lattice_df.loc[lattice_df['dipole']==0,'dipole'] = np.nan

In [ ]:
lattice_df

In [ ]:
collider['lhcb1'].element_dict['mb.b8r3.b1..2'].

In [ ]:
len(all_mb)

In [ ]:
lattice_df

In [ ]:
collider['lhcb1'].element_dict['mbxw.d4l5']

In [ ]:
[name for ee,name in zip(collider['lhcb1'].elements,collider['lhcb1'].element_names) if type(ee) is xt.elements.Multipole]

In [ ]:
lattice_df

In [ ]:
len(tw)

In [ ]:
[name for ee,name in zip(collider['lhcb1'].elements,collider['lhcb1'].element_names) if 'mb.' in name]

In [ ]:
lattice_df.dipole/lattice_df.dipole

In [ ]:
_knl

In [ ]:
lattice_df.dipole

In [ ]:
np.isnan(lattice_df.dipole)

In [ ]:
import matplotlib.pyplot as plt


plt.figure()
plt.plot(lattice_df.s,lattice_df.dipole/lattice_df.dipole,'-')
plt.fill_between(lattice_df.s,y1=-np.ones(len(lattice_df.s)),y2=lattice_df.dipole/lattice_df.dipole)
# plt.xlim([19600,19800])

In [ ]:
lattice

In [ ]:
np.pad(a.astype(float), (0, 3-len(a)), 'constant', constant_values=np.nan)

In [ ]:
collider['lhcb1'].element_dict['mqwa.b4r3.b1..1']

In [ ]:
a = np.array([1])[:3]
np.pad(a, (0, 3-len(a)), 'constant', constant_values=0)

In [ ]:
collider['lhcb1'].element_dict['mb.c33l4.b1..2'].knl

In [ ]:
collider['lhcb1'].element_dict['mcs.c33l4.b1']

In [ ]:
collider['lhcb1'].elements[3] is xt.elements.Multipole

In [ ]:
collider['lhcb1'].element_names




In [ ]:

# importing show from
# bokeh.io to show the plot
from bokeh.io import show
 
# importing legend from bokeh.models
from bokeh.models import Legend
 
# importing figure from bokeh.plotting
from bokeh.plotting import figure
 
# importing numpy package from python
import numpy as np
 
# Creating an empty figure of plot height=600
# and plot width=600
fig = figure()
 
# Creating point1 which is a yellow color line with
# line width as 3 for a set of points
point1 = fig.line(x=[0, 1.43, 2.76, 3.24, 4.45, 5.65,
                     6.98, 7, 8, 9.76, 10.67, 11.54, 12.567,
                     13.21, 14.65, 15, 16.45, 17, 18.32, 19.57, 20],
                  y=np.linspace(0, 2, 21),
                  line_width=3, color="yellow")
 
# Creating point2 which is a purple color line with
# line width as 2 for a set of points
point2 = fig.line(x=np.linspace(0, 20, 20), y=np.linspace(0, 3, 20),
                  line_width=2, color="purple")
 
# Creating point3 which is a pink color line with
# line width as 4 for a set of points
point3 = fig.line(x=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                     12, 13, 14, 15, 16, 17, 18, 19, 20],
                  y=[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2.5, 2.34,
                     2.48, 2.67, 2.12, 2.01, 1.67, 1.98, 1.07, 1],
                  line_width=4, color="pink")
 
# Creating point4 which is a green color line with
# line width as 2 for a set of points
point4 = fig.line(x=[0, 20], y=5,
                  line_width=2,
                  color="green")
 
# Using legend we are placing two point descriptions
# beside each other horizontally
legend1 = Legend(items=[("point1", [point1]),
                        ("point2", [point2])],
                 location=(7, 2), orientation="horizontal")
 
# Using legend we are placing the other two point
# descriptions beside each other horizontally
legend2 = Legend(items=[("point3", [point3]),
                        ("point4", [point4])],
                 location=(7, 2), orientation="horizontal")
 
# placing legend1 at the top
fig.add_layout(legend1, 'above')
 
# placing legend2 at the top
fig.add_layout(legend2, 'above')
 
# showing the figure
show(fig)